In [3]:
import rosbag
PATH="/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/tro2020-bags/pre/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
input_bag = PATH +"drone1-swarm_local_2020-12-13-21-31-51.bag"
output_bag = PATHOUT + "drone1-swarm_local_2020-12-13-21-31-51_processed.bag"
moveout_topics = {
    "/swarm_drones/node_detected",
    "/swarm_drones/swarm_drone_fused",
    "/swarm_drones/swarm_drone_fused_relative",
    "/swarm_drones/swarm_frame",
    "/swarm_drones/swarm_frame_predict"
}
#Moveout node_detected
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        if topic in moveout_topics:
            pass
        else:
            #outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)
            outbag.write(topic, msg, t)            